In [ ]:
# Si no tenemos el módulo instalado
#!pip install langchain langchain-openai

### LangChain: Modelos, Prompts and Parsers de salida


In [1]:
# Primero importamos las bibliotecas necesarias:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

from dotenv import load_dotenv
import os

In [2]:
# Cargar variables de entorno
load_dotenv()
# Configurar el motor de OpenAI
engine = "gpt-3.5-turbo"
api_key=os.getenv("OPENAI_API_KEY")

In [3]:
chat = ChatOpenAI(temperature=0.7, model=engine,openai_api_key=api_key)

## Plantilla de Prompt

In [4]:
estilo_formal = """Español Castellano \
correcto, educado y conciliador
"""

In [5]:
plantilla = """Transforma el texto \
delimitado por acento grave triple \
en un texto con el estilo {estilo}. \
text: ```{texto}```
"""

In [9]:
prompt_template = ChatPromptTemplate.from_template(plantilla)
#prompt_template.messages[0].prompt
#prompt_template.messages[0].prompt.input_variables

In [10]:
texto_original = """
Bueno, la verdad es que paso bastante de los resultados.
Me importa un bledo lo que piensen en dirección. La próxima ver que me contacten
para hablar de esto, les voy a mandar al carajo!
"""

In [11]:
messages = prompt_template.format_messages(
                    estilo=estilo_formal,
                    texto=texto_original)

In [12]:
print(type(messages))
print(type(messages[0]))
print(messages[0])

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>
content='Transforma el texto delimitado por acento grave triple en un texto con el estilo Español Castellano correcto, educado y conciliador\n. text: ```\nBueno, la verdad es que paso bastante de los resultados.\nMe importa un bledo lo que piensen en dirección. La próxima ver que me contacten\npara hablar de esto, les voy a mandar al carajo!\n```\n'


In [13]:
###Llamamos al LLM para transformar el mensaje según la plantilla

response = chat.invoke(messages)
print(response)

content='Bueno, la verdad es que no le doy mucha importancia a los resultados. No me preocupa en absoluto lo que piensen en dirección. La próxima vez que me contacten para tratar este asunto, les manifestaré mi desacuerdo de una manera más educada.'


### Procesador de salida (output parsers)
Ayudan a transformar la salida cruda del modelo en algo más utilizable o en un formato específico que sea más fácil de manejar en la aplicación donde se utiliza el modelo.

In [14]:
{
  "difícil": False,
  "profesor": "Muy bueno!",
  "calidad_precio": "buena relación calidad-precio"
}

{'difícil': False,
 'profesor': 'Muy bueno!',
 'calidad_precio': 'buena relación calidad-precio'}

In [15]:
valoracion = """\
Me ha gustado mucho el curso. El profesor ha sido muy profesional
y se notaba su experiencia y grandes conocimientos.
A pesar de ello, era la primera vez que estudiaba estos conceptos
y me costó un poco asimilarlos.
Estoy muy contento con haber hecho esta inversión de dinero en mi conocimiento.
Lo recomendaré a otros colegas de trabajo!
"""

plantilla = """\
Del siguiente texto, extrae la siguiente información:

dificultad: El curso le resulto fácil o difícil
Contesta True si fue difícil, False si no lo fue o no se indica.

profesor: El profesor cumplió con sus expectativas?
Valoralo el sentimiento sobre el profesor del 1 al 5.
Si no se menciona nada sobre el profesor devuelve -1.

calidad_precio: Extrae cualquier referencia a la calidad o precio,\
y relacionalos ofreciendo el sentimiento.

Formatea la salida como JSON con las siguientes claves:
dificultad
profesor
calidad_precio

text: {text}
"""

In [16]:
from langchain.prompts import ChatPromptTemplate

prompt_plantilla = ChatPromptTemplate.from_template(plantilla)
print(prompt_plantilla)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='Del siguiente texto, extrae la siguiente información:\n\ndificultad: El curso le resulto fácil o difícil\nContesta True si fue difícil, False si no lo fue o no se indica.\n\nprofesor: El profesor cumplió con sus expectativas?\nValoralo el sentimiento sobre el profesor del 1 al 5.\nSi no se menciona nada sobre el profesor devuelve -1.\n\ncalidad_precio: Extrae cualquier referencia a la calidad o precio,y relacionalos ofreciendo el sentimiento.\n\nFormatea la salida como JSON con las siguientes claves:\ndificultad\nprofesor\ncalidad_precio\n\ntext: {text}\n'))]


In [19]:
messages = prompt_plantilla.format_messages(text=valoracion)

chat = ChatOpenAI(temperature=0.7, openai_api_key=api_key)
response = chat.invoke(messages)
print(response.content)

{
  "dificultad": true,
  "profesor": 5,
  "calidad_precio": "inversión de dinero"
}


In [18]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [20]:
dificultad_schema = ResponseSchema(name="dificultad", description="¿El curso le resulto fácil o difícil? Contesta True si fue difícil, False si no lo fue o no se indica.")
profesor_schema = ResponseSchema(name="profesor", description="El profesor cumplió con sus expectativas? Valoralo el sentimiento sobre el profesor del 1 al 5. Si no se menciona nada sobre el profesor devuelve -1.")
calidad_precio_schema = ResponseSchema(name="calidad_precio", description="Extrae cualquier referencia a la calidad o el precio y relaciónalos ofreciendo el sentimiento.")

response_schemas = [dificultad_schema,
                    profesor_schema,
                    calidad_precio_schema]

In [21]:
procesador_salida = StructuredOutputParser.from_response_schemas(response_schemas)

In [22]:
format_instructions = procesador_salida.get_format_instructions()

In [23]:
plantilla2 = """\
Del siguiente texto, extrae la siguiente información:

dificultad: El curso le resulto fácil o difícil. Contesta True si fue difícil, False si no lo fue o no se indica.

profesor: El profesor cumplió con sus expectativas? Valoralo el sentimiento sobre el profesor del 1 al 5. Si no se menciona nada sobre el profesor devuelve -1.

calidad_precio: Extrae cualquier referencia a la calidad o precio y relacionalos ofreciendo el sentimiento.

Formatea la salida como JSON con las siguientes claves:
dificultad
profesor
calidad_precio

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=plantilla2)

messages = prompt.format_messages(text=valoracion, format_instructions=format_instructions)

In [24]:
print(messages[0].content)

Del siguiente texto, extrae la siguiente información:

dificultad: El curso le resulto fácil o difícil. Contesta True si fue difícil, False si no lo fue o no se indica.

profesor: El profesor cumplió con sus expectativas? Valoralo el sentimiento sobre el profesor del 1 al 5. Si no se menciona nada sobre el profesor devuelve -1.

calidad_precio: Extrae cualquier referencia a la calidad o precio y relacionalos ofreciendo el sentimiento.

Formatea la salida como JSON con las siguientes claves:
dificultad
profesor
calidad_precio

text: Me ha gustado mucho el curso. El profesor ha sido muy profesional
y se notaba su experiencia y grandes conocimientos.
A pesar de ello, era la primera vez que estudiaba estos conceptos
y me costó un poco asimilarlos.
Estoy muy contento con haber hecho esta inversión de dinero en mi conocimiento.
Lo recomendaré a otros colegas de trabajo!


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```js

In [25]:
response = chat.invoke(messages)

In [26]:
print(response.content)

```json
{
	"dificultad": true,
	"profesor": 5,
	"calidad_precio": "positivo"
}
```


In [27]:
output_dict = procesador_salida.parse(response.content)

In [28]:
output_dict

{'dificultad': True, 'profesor': 5, 'calidad_precio': 'positivo'}